In [5]:
from __future__ import print_function
import torch.utils.data as data
from PIL import Image
import os
import os.path
import errno
import torch
import codecs


class fashion(data.Dataset):
    """`MNIST <http://yann.lecun.com/exdb/mnist/>`_ Dataset.
    Args:
        root (string): Root directory of dataset where ``processed/training.pt``
            and  ``processed/test.pt`` exist.
        train (bool, optional): If True, creates dataset from ``training.pt``,
            otherwise from ``test.pt``.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
    """
    urls = [
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz',
    ]
    raw_folder = 'raw'
    processed_folder = 'processed'
    training_file = 'training.pt'
    test_file = 'test.pt'

    def __init__(self, root, train=True, transform=None, target_transform=None, download=True):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.train = train 

        if download:
            self.download()

        if not self._check_exists():
            raise RuntimeError('Download=True set')

        if self.train:
            self.train_data, self.train_labels = torch.load(
                os.path.join(root, self.processed_folder, self.training_file))
        else:
            self.test_data, self.test_labels = torch.load(os.path.join(root, self.processed_folder, self.test_file))

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]

        img = Image.fromarray(img.numpy(), mode='L')

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        if self.train:
            return len(self.train_data)
        else:
            return len(self.test_data)

    def _check_exists(self):
        return os.path.exists(os.path.join(self.root, self.processed_folder, self.training_file)) and \
            os.path.exists(os.path.join(self.root, self.processed_folder, self.test_file))

    def download(self):

        from six.moves import urllib
        import gzip

        if self._check_exists():
            return

        try:
            os.makedirs(os.path.join(self.root, self.raw_folder))
            os.makedirs(os.path.join(self.root, self.processed_folder))
        except OSError as e:
            if e.errno == errno.EEXIST:
                pass
            else:
                raise

        for url in self.urls:
            print('Downloading ' + url)
            data = urllib.request.urlopen(url)
            filename = url.rpartition('/')[2]
            file_path = os.path.join(self.root, self.raw_folder, filename)
            with open(file_path, 'wb') as f:
                f.write(data.read())
            with open(file_path.replace('.gz', ''), 'wb') as out_f, \
                    gzip.GzipFile(file_path) as zip_f:
                out_f.write(zip_f.read())
            os.unlink(file_path)


        print('Processing...')

        training_set = (
            read_image_file(os.path.join(self.root, self.raw_folder, 'train-images-idx3-ubyte')),
            read_label_file(os.path.join(self.root, self.raw_folder, 'train-labels-idx1-ubyte'))
        )
        test_set = (
            read_image_file(os.path.join(self.root, self.raw_folder, 't10k-images-idx3-ubyte')),
            read_label_file(os.path.join(self.root, self.raw_folder, 't10k-labels-idx1-ubyte'))
        )
        with open(os.path.join(self.root, self.processed_folder, self.training_file), 'wb') as f:
            torch.save(training_set, f)
        with open(os.path.join(self.root, self.processed_folder, self.test_file), 'wb') as f:
            torch.save(test_set, f)

        print('Done!')


def get_int(b):
    return int(codecs.encode(b, 'hex'), 16)


def parse_byte(b):
    if isinstance(b, str):
        return ord(b)
    return b


def read_label_file(path):
    with open(path, 'rb') as f:
        data = f.read()
        assert get_int(data[:4]) == 2049
        length = get_int(data[4:8])
        labels = [parse_byte(b) for b in data[8:]]
        assert len(labels) == length
        return torch.LongTensor(labels)


def read_image_file(path):
    with open(path, 'rb') as f:
        data = f.read()
        assert get_int(data[:4]) == 2051
        length = get_int(data[4:8])
        num_rows = get_int(data[8:12])
        num_cols = get_int(data[12:16])
        images = []
        idx = 16
        for l in range(length):
            img = []
            images.append(img)
            for r in range(num_rows):
                row = []
                img.append(row)
                for c in range(num_cols):
                    row.append(parse_byte(data[idx]))
                    idx += 1
        assert len(images) == length
        return torch.ByteTensor(images).view(-1, 28, 28)
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

train_dataset = fashion(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = fashion(root='./data',
                           train=False,
                           transform=transforms.ToTensor())


batch_size = 100
n_iters = 5000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        #MODEL:
        ##Conv layers: Conv2D, relu activation, batch norm, max pooling
        ##Fully Connected layer: linear layer

        self.cnn1= nn.Conv2d(in_channels=1, out_channels= 16, kernel_size=5, stride=1, padding=2)

        self.relu1 = nn.ReLU()
        self.conv1_bn = nn.BatchNorm2d(16)

        self.MaxPool1 = nn.MaxPool2d(kernel_size=2)

        self.cnn2 = nn.Conv2d(in_channels= 16, out_channels= 32, kernel_size=5,stride=1, padding=2)
        self.relu2 = nn.ReLU()
        self.conv2_bn = nn.BatchNorm2d(32)
        
        self.MaxPool2 = nn.MaxPool2d(kernel_size=2)
        
        self.cnn3 = nn.Conv2d(in_channels= 32, out_channels= 64, kernel_size=5,stride=1, padding=2)
        self.relu3 = nn.ReLU()
        self.conv3_bn = nn.BatchNorm2d(64)
        
        self.MaxPool3 = nn.MaxPool2d(kernel_size=2)
        
        self.fc1 = nn.Linear(576, 10)
    
    
    def forward(self, x):
        
        out = self.cnn1(x)
        out= self.relu1(out)
        our = self.conv1_bn(out)
        
        out= self.MaxPool1(out)
        
        out= self.cnn2(out)
        out = self.relu2(out)
        out = self.conv2_bn(out)
        
        out = self.MaxPool2(out)
        
        out= self.cnn3(out)
        out = self.relu3(out)
        out = self.conv3_bn(out)
        
        out = self.MaxPool3(out)
        
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        
        return out

model = CNNModel()

criterion = nn.CrossEntropyLoss()

learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


iter = 0

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        iter +=1
        
        if iter%100 ==0:
            correct= 0
            total=0
            
            for images, labels in test_loader:
                images = Variable(images)
                
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                    
                total += labels.size(0)
                    
                correct += (predicted==labels).sum()
        
            accuracy = 100*correct/total
            
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

torch.save(model.state_dict(), 'cnn.pkl')

Iteration: 100. Loss: 0.46369311213493347. Accuracy: 80.51
Iteration: 200. Loss: 0.4890281558036804. Accuracy: 81.89
Iteration: 300. Loss: 0.49713799357414246. Accuracy: 85.18
Iteration: 400. Loss: 0.5358873605728149. Accuracy: 84.63
Iteration: 500. Loss: 0.44168487191200256. Accuracy: 86.25
Iteration: 600. Loss: 0.3038049638271332. Accuracy: 86.61
Iteration: 700. Loss: 0.26195546984672546. Accuracy: 87.29
Iteration: 800. Loss: 0.21691088378429413. Accuracy: 87.15
Iteration: 900. Loss: 0.49018216133117676. Accuracy: 87.09
Iteration: 1000. Loss: 0.27443867921829224. Accuracy: 87.77
Iteration: 1100. Loss: 0.284212589263916. Accuracy: 87.55
Iteration: 1200. Loss: 0.35525181889533997. Accuracy: 87.04
Iteration: 1300. Loss: 0.29659757018089294. Accuracy: 88.35
Iteration: 1400. Loss: 0.26283472776412964. Accuracy: 87.55
Iteration: 1500. Loss: 0.399793803691864. Accuracy: 88.58
Iteration: 1600. Loss: 0.2412887066602707. Accuracy: 88.32
Iteration: 1700. Loss: 0.16593511402606964. Accuracy: 88.